# myTravel Recommender System

## Import Library

In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import pathlib
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate



2023-06-04 14:18:12.565688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 14:18:12.758481: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-04 14:18:12.758525: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-04 14:18:12.810829: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-04 14:18:14.273140: W tensorflow/stream_executor/pla

## Load Data

In [2]:
# Pembuatan Dataframe
tourism_df = pd.read_csv("dataset/tourism_with_id.csv")
rating_df = pd.read_csv("dataset/tourism_rating.csv")

## Data Preparation

In [3]:
# Feature yang dipakai saja
data_place = tourism_df[["Place_Id", "Place_Name", "Category", "Rating"]]

In [4]:
# Jumlah fitur
num_cat = len(data_place["Category"].unique())
num_user = len(rating_df["User_Id"].unique())


In [5]:
def gen_user_vecs(user_vecs, num):
  vec = []

  for i in range(num):
    vec.append(user_vecs)

  return np.array(vec)

In [6]:
data_place.head()

,Place_Id,Place_Name,Category,Rating
0,1,Monumen Nasional,Budaya,4.6
1,2,Kota Tua,Budaya,4.6
2,3,Dunia Fantasi,Taman Hiburan,4.6
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan,4.5
4,5,Atlantis Water Adventure,Taman Hiburan,4.5


In [6]:
rating_df.head()

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


## Content-Based Filtering

In [7]:
# Preparing place_rating dataset
place_prep = rating_df.copy()
place_prep = place_prep.drop("User_Id", axis=1)
place_prep = place_prep.groupby(['Place_Id']).count()
place_prep = place_prep.reset_index()


place_rating = {'Place_Id': data_place["Place_Id"].unique()}
place_rating = pd.DataFrame(place_rating)
cols_name = data_place["Category"].unique()

for i in range(num_cat):
  place_rating.insert(i+1, cols_name[i], np.zeros(len(data_place["Place_Id"].unique())), True)

for i in range(len(data_place)):
  place_rating[data_place["Category"][i]][i] = 1

place_rating.insert(7, "Rating", data_place["Rating"], True)
place_rating

/tmp/ipykernel_84/3071785415.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  place_rating[data_place["Category"][i]][i] = 1


,Place_Id,Budaya,Taman Hiburan,Cagar Alam,Bahari,Pusat Perbelanjaan,Tempat Ibadah,Rating
0,1,1.0,0.0,0.0,0.0,0.0,0.0,4.6
1,2,1.0,0.0,0.0,0.0,0.0,0.0,4.6
2,3,0.0,1.0,0.0,0.0,0.0,0.0,4.6
3,4,0.0,1.0,0.0,0.0,0.0,0.0,4.5
4,5,0.0,1.0,0.0,0.0,0.0,0.0,4.5
...,...,...,...,...,...,...,...,...
432,433,1.0,0.0,0.0,0.0,0.0,0.0,4.4
433,434,0.0,1.0,0.0,0.0,0.0,0.0,4.6
434,435,0.0,1.0,0.0,0.0,0.0,0.0,4.4
435,436,0.0,1.0,0.0,0.0,0.0,0.0,4.6


In [8]:
# Preparing user rating dataset
user_prep = pd.merge(rating_df, data_place)
user_prep = user_prep.drop(["Rating", "Place_Id"], axis=1)
user_prep = user_prep.groupby(['User_Id', 'Category']).mean()
user_prep = user_prep.reset_index()

user_rating = {'User_Id': rating_df["User_Id"].unique()}
user_rating = pd.DataFrame(user_rating)
col_name = data_place["Category"].unique()

for i in range(num_cat):
  user_rating.insert(i+1, col_name[i], np.array([3.0]*num_user), True)

for i in range(len(user_prep)):
  user_rating[user_prep["Category"][i]][user_prep["User_Id"][i]-1] = user_prep["Place_Ratings"][i].copy()

user_rating

/tmp/ipykernel_84/3316091842.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_rating[user_prep["Category"][i]][user_prep["User_Id"][i]-1] = user_prep["Place_Ratings"][i].copy()


,User_Id,Budaya,Taman Hiburan,Cagar Alam,Bahari,Pusat Perbelanjaan,Tempat Ibadah
0,1,3.500000,3.800000,3.444444,2.000000,3.0,2.0
1,2,3.444444,3.500000,2.800000,2.000000,3.0,4.0
2,3,3.230769,2.900000,4.000000,4.333333,3.0,5.0
3,4,3.083333,3.500000,3.750000,5.000000,4.0,3.0
4,5,3.285714,3.461538,2.888889,3.500000,5.0,5.0
...,...,...,...,...,...,...,...
295,296,3.000000,3.100000,2.800000,3.000000,2.0,2.5
296,297,3.000000,4.000000,3.142857,3.333333,1.0,2.5
297,298,3.833333,4.200000,3.428571,2.750000,5.0,2.0
298,299,3.230769,2.600000,2.250000,2.000000,2.5,4.0


In [9]:
# Making place_rating and user_rating the same length with rating_df

place_ratings = rating_df.merge(place_rating, how="left", on="Place_Id")
place_ratings = place_ratings.drop(["User_Id", "Place_Id", "Place_Ratings"], axis=1)

user_ratings = rating_df.merge(user_rating, how="left", on="User_Id")
user_ratings = user_ratings.drop(["User_Id", "Place_Id", "Place_Ratings"], axis=1)

y = np.array(rating_df["Place_Ratings"])

In [10]:
# scale training data

scalerItem = StandardScaler()
scalerItem.fit(place_ratings)
place_train = scalerItem.transform(place_ratings)

scalerUser = StandardScaler()
scalerUser.fit(user_ratings)
user_train = scalerUser.transform(user_ratings)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y.reshape(-1, 1))
y_train = scalerTarget.transform(y.reshape(-1, 1))


In [11]:
# Split to train and test

place_train, place_test = train_test_split(place_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"place training data shape: {user_train.shape}")
print(f"place test data shape: {user_test.shape}")

place training data shape: (8000, 6)
place test data shape: (2000, 6)


In [12]:
# Modelling
num_outputs = 16

tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
  
  
])

item_NN = tf.keras.models.Sequential([   
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_cat))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_cat+1))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

2023-06-04 14:18:50.868058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-04 14:18:50.868125: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-04 14:18:50.868147: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-3c85a2c7-57f1-4402-bc51-41180ba04a5d): /proc/driver/nvidia/version does not exist
2023-06-04 14:18:50.868518: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Model: "model"
_____________________________________________________________

In [13]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam()
model.compile(optimizer=opt,
              loss=cost_fn)
model.fit([user_train, place_train], y_train, epochs=50)

Epoch 1/50
250/250 [==============================] - 2s 4ms/step - loss: 0.4467
Epoch 2/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4248
Epoch 3/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4171
Epoch 4/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4141
Epoch 5/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4104
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4096
Epoch 7/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4074
Epoch 8/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4069
Epoch 9/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4056
Epoch 10/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4047
Epoch 11/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4042
Epoch 12/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4041
Epoch 13/50
250/250 [====

In [14]:
model.evaluate([user_test, place_test], y_test)

63/63 [==============================] - 0s 2ms/step - loss: 0.4309


0.4309030771255493

## Test Model

In [15]:
# Predictions for New User

rate_Budaya = 3
rate_Taman_Hiburan = 3
rate_Cagar_Alam = 3
rate_Bahari = 3
rate_Pusat_Perbelanjaan	= 3
rate_Tempat_Ibadah = 3

user_vec = np.array([rate_Budaya, rate_Taman_Hiburan, rate_Cagar_Alam,
                      rate_Bahari, rate_Pusat_Perbelanjaan, rate_Tempat_Ibadah])

# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(place_rating))



# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(place_rating.drop("Place_Id", axis=1))

# make a prediction
y_p = model.predict([suser_vecs, sitem_vecs])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)


# sort the results, highest prediction first
sorted_index = np.argsort(-(y_pu),axis=0).reshape(-1).tolist() #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = data_place.copy()
sorted_items = sorted_items.loc[sorted_index,:]  #using unscaled vectors for display
sorted_items.insert(4, "yp", sorted_ypu.flatten())
sorted_items.set_index("Place_Id").head(20)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
14/14 [==============================] - 0s 4ms/step


,Place_Name,Category,Rating,yp
Place_Id,,,,
122,Watu Goyang,Budaya,4.4,3.257875
265,Museum Mandala Wangsit Siliwangi,Budaya,4.4,3.257875
42,Museum Seni Rupa dan Kramik,Budaya,4.4,3.257875
32,Setu Babakan,Budaya,4.4,3.257875
89,De Mata Museum Jogja,Budaya,4.4,3.257875
421,Museum Kesehatan Dr. Adhyatma,Budaya,4.4,3.257875
176,Museum Gunung Merapi,Budaya,4.4,3.257875
351,Old City 3D Trick Art Museum,Budaya,4.4,3.257875
416,Keraton Surabaya,Budaya,4.4,3.257875


In [17]:
len(user_vecs)

437

In [19]:
len(sitem_vecs)

437

In [49]:
# Predictions for Hidden Gems


# sort the results, highest prediction first
sorted_gems = data_place.copy()
sorted_gems = sorted_gems.loc[sorted_index,:]  #using unscaled vectors for display
sorted_gems.insert(4, "yp", sorted_ypu.flatten())
sorted_gems.insert(4, "Rating_Count", place_prep.drop("Place_Id", axis=1))
sorted_gems = sorted_gems.loc[sorted_gems["Rating_Count"] <= 20]
sorted_gems = sorted_gems.loc[sorted_gems["Rating"] >= 4.5]
sorted_gems.set_index("Place_Id").head(20)

,Place_Name,Category,Rating,Rating_Count,yp
Place_Id,,,,,
129,Bukit Lintang Sewu,Taman Hiburan,4.5,17,3.312435
28,Wisata Agro Edukatif Istana Susu Cibugary,Taman Hiburan,4.5,18,3.312435
128,Gumuk Pasir Parangkusumo,Taman Hiburan,4.5,20,3.312435
214,Trans Studio Bandung,Taman Hiburan,4.5,18,3.312435
334,Chingu Cafe Little Seoul,Taman Hiburan,4.5,18,3.312435
205,Desa Wisata Tembi,Taman Hiburan,4.5,16,3.312435
399,Taman Pelangi,Taman Hiburan,4.5,17,3.312435
112,Bukit Bintang Yogyakarta,Taman Hiburan,4.5,17,3.312435
293,Taman Badak,Taman Hiburan,4.5,16,3.312435


## Convert Model to tflite Format

In [24]:
export_dir = "./recommender"
tf.saved_model.save(model, export_dir = export_dir)

INFO:tensorflow:Assets written to: ./recommender/assets


In [25]:
optimization = tf.lite.Optimize.DEFAULT
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [optimization]
tflite_model = converter.convert()

2023-06-04 10:40:06.579183: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-04 10:40:06.579231: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-04 10:40:06.579552: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./recommender
2023-06-04 10:40:06.587809: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-04 10:40:06.587855: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./recommender
2023-06-04 10:40:06.618817: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-06-04 10:40:06.791787: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./recommender
2023-06-04 10:40:06.881552: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 302005 microseco

In [29]:
tflite_model_file = pathlib.Path('./recommender.tflite')
tflite_model_file.write_bytes(tflite_model)

90968

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3c85a2c7-57f1-4402-bc51-41180ba04a5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>